# KOL Ascension Log Wrapper (v2.1)
_A Styling Project by Aaron M. (AKA Captain Scotch, #437479)_

Hey all! This is a wrapper meant to isolate and analyze Ascension logs via Python, formatting everything into a clean Excel wrapper. Originally, this was simply an overlay of [CKB's run log parser in ASH](https://kolmafia.us/showthread.php?22963-RunLogSummary), which is fantastic, but as I'm more of a Python dev this quickly enlarged into a broader project that turned into a personal goal to build a Python log parser. That's in development right now! I tried to make it as clean as possible; let me know if you have any questions.

In [1]:
import pandas as pd
import requests
import random
import numpy as np
import re

In [2]:
# Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
kolmafDir = '/Users/amcguire/Library/Application Support/KoLmafia/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
kolName   = 'monsieur bob'
runDates  = ['20190503'] 
runNotes  = 'blah blah blah blah /n blah blah blah BLAH /n blah blah, blah blah, Baaaaaah'

In [427]:
# Read in the last parsed run via RunLogSum.ash, for testing purposes
oldRun = pd.read_csv('{}data/{}-runlog_0116.txt'.format(kolmafDir,kolName),sep='\t')

In [3]:
# These are, all things considered, extremely small files. So I just read the whole
#   thing into memory so that it's all accessible without finicky itertools stuff.

# Populate our session dictionary
dDict = {}

for day, session in enumerate(runDates):
    # Python is zero-indexed, so we increment day by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    dDict[day+1] = open('{}sessions/{}_{}.txt'.format(kolmafDir, kolName.replace(" ","_"), session)).read()
    
    # Remove everything before Valhalla & after freeing Ralph
    if day == 0:
        start = dDict[1].find('Welcome to Valhalla!')
        dDict[1] = dDict[1][start:]
        
    else:
        end   = dDict[day+1].find('Freeing King Ralph')
        dDict[day+1] = dDict[day+1][:end]

# Also, to minimize server hits, grab a few files from the Mafia SVN data

#  - mafEncounters = can be used to identify clover & semirares
#  - mafCombats    = combats w/ mafia; can use to figure out combat rate & monster buckets in each zone
#  - mafMonster    = monster names/IDs & stats, as well as drops!
#  - mafItems      = all items in the game, can be used to cross-ref w/ monsterdrops
#  - mafMods       = all modifiers in the game and what they actually do
#  - mafBounty     = all the bounties in the game 

mafEncounters = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/encounters.txt')
mafCombats = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/combats.txt')
mafMonster = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/monsters.txt')
mafItems   = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/items.txt')
mafMods    = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/modifiers.txt')

# Random stuff I might not end up using

mafBounty  = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/bounty.txt')

In [429]:
# This is from an effort a while ago to figure out how to build a script that knows where things "should" be.

yolo = {}
for x, line in enumerate(mafEncounters.text.split('\n')):
    try: 
        temp = line.split('\t')
        if len(temp) == 3:
            yolo[x] = {'LOCATION':temp[0], 'ADVTYPE':temp[1], 'NAME':temp[2]}
    except:
        print('error @ {}'.format(x))
    

In [430]:
findString = 'round 15'
buffer     = 3
day        = 1

# simple print w/ buffer for finding things within the logs
for ct, x in enumerate(dDict[day].split(sep='\n')):
    if findString.lower() in x.lower(): 
        print('===========\n'+
              '\n'.join(dDict[day].split(sep='\n')[(ct-buffer):(ct+buffer)])+
              '\n===========\n')

Round 13: Captain Scotch uses the seal tooth!
Round 14: fantasy bandit takes 1 damage.
Round 14: Captain Scotch uses the seal tooth!
Round 15: fantasy bandit takes 1 damage.
Round 15: Captain Scotch uses the seal tooth!
Round 16: fantasy bandit takes 1 damage.

Round 14: fantasy bandit takes 1 damage.
Round 14: Captain Scotch uses the seal tooth!
Round 15: fantasy bandit takes 1 damage.
Round 15: Captain Scotch uses the seal tooth!
Round 16: fantasy bandit takes 1 damage.
Round 16: Captain Scotch uses the seal tooth!

Round 13: Captain Scotch attacks!
Round 14: You lose 21 hit points
Round 14: Captain Scotch attacks!
Round 15: rock scorpion takes 22 damage.
Round 15: Captain Scotch wins the fight!
You gain 928 Meat

Round 14: You lose 21 hit points
Round 14: Captain Scotch attacks!
Round 15: rock scorpion takes 22 damage.
Round 15: Captain Scotch wins the fight!
You gain 928 Meat
You acquire an item: handful of sand

Round 13: Captain Scotch attacks!
Round 14: ancient protector spirit 

In [431]:
def monsterLocation(mon):
    ''' Sub-function that uses KOLMafia's combats directory to ascertain
        where a monster is likely to occur (used for establishing wanderer
        likelihood in the absence of full adventure parsing; was used for
        initial tests of digitize capture). Eventually will use this for
        full error-checking of the parser. '''
    
    locs = []
    
    mon = str(mon).strip()
    
    # Read in combats.txt and reformat it via split
    for line in mafCombats.text.split('\n'):
        if   mon in line: 
            locs = locs + [line.split('\t')[0]]
            
        # For some reason, Camel's Toe shows up as "The" Camel's Toe when
        #   you genie wish; this checks for and removes extraneous "The"
        elif mon.lower().replace('the ','') in line.lower():
                locs = locs + [line.split('\t')[0]]
    
    if locs == []: locs = ['FOOTAGE NOT FOUND']
        
    return locs
    
# gross this doesn't work without internet either
# monsterLocation('Green Ops Soldier')

In [432]:
def cleanNames(monName):
    ''' This won't be much of an issue going forward, but
        the Intergnat had an annoying habit of modifying 
        monster names. This attempts to clean that particular 
        detritus to extract pure names. I'm assuming this
        will come back in other forms as well.
        
        Features to add: 
        
            - parse OCRS modifiers 
            - parse "Yes, Can Has" effect'''
    
    # All the strings added to monster names. Haven't added 
    #   OCRS modifiers yet, but intend to. Also would like 
    #   to include rewriting the can has skill, in case it
    #   comes up through the crazy horse, but haven't yet
    
    gnatList = ['NAMED NEIL', 'ELDRITCH HORROR ', 'AND TESLA',
                'WITH BACON!!!', 'WITH SCIENCE']
    
    ocrsMods = []
    
    replList = gnatList + ocrsMods
    
    for repl in replList:
        monName = monName.replace(repl,'')
        
    # Insert renaming for that one cheeseburger skill here
    
    return monName.strip()

In [433]:
def extractWanderers(sessionLogs):
    ''' Quick yet annoying function to attempt to ascertain wanderers. The 
        Digitize parser (unfortunately) doesn't work particularly well yet;
        when ascertaining digitize usage I usually just look over the print &
        mental math for the spreadsheet. The enamorang parser works better,
        which is good, since it's currently the only standard "choice" 
        wanderer as of 1/19. 
        
        Features to add: 
        
            - include detection of vote wanderers 
            - once the XML turn parser is done, revise this to reference it 
            - (you basically fuckin did that already in this hacky POS) 
            - doesn't properly handle LOV over two days '''
    
    wandererDict = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        wandererDict[day] = {}
        
        # Tracking digitized monsters is a pain in the ass. Luckily, simply
        #   extracting who they are can be pretty easy with some rough 
        #   nested loops.
        
        digiMons = []
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'casts DIGITIZE!' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                digiMons = digiMons + [monName]
    
        for mon in digiMons:
            
            # Now that we have that list, we want to figure out how many times
            #   they were fought. To do this, we'll need to compare the zone
            #   they originated from to the zones they have been found in.
            #   This method is meh, but kind of works, in the absence of Mafia 
            #   saving intro messages for digitized monsters. 
            
            monLoc = monsterLocation(mon)
            
            # So, cool unintentional thing here; when Mafia logs Witchess, it
            #   totally messes up the turn/combat statement, because it treats
            #   it differently. This is great! It means digitized Witchess
            #   pieces are super easy to find.
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        print('  Could not parse {}'.format(data))
    
        monLOVs = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'uses the LOV Enamorang' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                        
                        try: 
                            turn = int(dataSplit[0][1:])
                            loct = dataSplit[1]
                        except:
                            turn = -999
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                monLOVs[turn] = monName
                
        for turn, mon in monLOVs.items():
            
            # Similar process for above, with a few small items. Here, I am
            #   checking turn differential to make positively sure it is the
            #   enamorang'd monster; it has to be 14 turns! 
            
            monLoc = monsterLocation(mon)
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        foundTurn = int(data[0][1:])
                        currLoct = data[1]
                        # For testing
                        #print('Turn {}: fought {} @ {}.'.format(foundTurn,mon,loct))
                    except:
                        # Warn that something was unparsable 
                        print('  Could not parse {}'.format(data))
                    
                    if turn != foundTurn:
                        wandererDict[day][mon] = {'Type':'Enamorang',
                                         'Location':currLoct,
                                         'Turn':foundTurn}
    
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wandererDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wandererDict.keys()],sort=False).T
    
extractWanderers(dDict)

Empty DataFrame
Columns: []
Index: []

In [434]:
def extractFreeKills(sessionLogs):
    ''' Quick function to ascertain freekills '''
    
    

In [435]:
def extractBanishesAndSniffs(sessionLogs, out='banishes'):
    ''' Quick function to ascertain banishes, sniffs, & macrometeorites '''
    
    banishList = ['ASDON MARTIN: SPRING-LOADED FRONT BUMPER!', 
                  'REFLEX HAMMER!', 
                  'KGB TRANQUILIZER DART!',
                  'THROW LATTE ON OPPONENT!',
                  'BALEFUL HOWL!']
    
    sniffList = ['OFFER LATTE TO OPPONENT!',
                 'PERCEIVE SOUL!']
    
    checkList = banishList + sniffList + ['MACROMETEORITE!']
    
    banishDict = {}
    sniffDict  = {}
    macroDict  = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        banishDict[day] = {}
        sniffDict[day]  = {}
        macroDict[day]  = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            for cond in checkList:
                if cond in row:
                    
                    if cond == 'MACROMETEORITE!':
                        nextMon = sessionLogs[day].split('\n')[ct+1].split('becomes')[1]
                    
                    x = 0

                    # Walk backwards through the file w/ a while loop 
                    while x < ct:
                        x +=1
                        monName = sessionLogs[day].split('\n')[ct-x]
                        if 'Encounter:' in monName:
                            dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                            try: 
                                turn = int(dataSplit[0][1:])
                                loct = dataSplit[1]
                            except:
                                turn = -999
                            break

                    monName = cleanNames(monName.replace('Encounter: ',''))
                    
                    if cond in banishList:
                        try:    num = max(list(banishDict[day].keys()))+1
                        except: num = 1
                        banishDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn}
                        
                    elif cond in sniffList:
                        try:    num = max(list(sniffDict[day].keys()))+1
                        except: num = 1
                        sniffDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn}
                        
                    else:
                        try:    num = max(list(macroDict[day].keys()))+1
                        except: num = 1
                        macroDict[day][num] = {'Monster':monName,'Type':cond,'Location':loct,'Turn':turn,'NextMon':nextMon}
        
    if out=="sniffs":
        outDict = sniffDict
    elif out=="macros":
        outDict = macroDict
    else:
        outDict = banishDict
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in outDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in outDict.keys()],sort=False).T


In [436]:
extractBanishesAndSniffs(dDict,'banishes')

Day #1  \
                                            Monster   
1                                  bouncing bookbat   
2                           Wardr&ouml;b nightstand   
3                              bouncing senile lihc   
4                                fragile slick lihc   
5                           huge sabre-toothed goat   
6                              throbbing drunk goat   
7                                       Protagonist   
8                            MagiMechTech MechaMech   
9                        artisanal gluttonous ghuol   
10                  jittery possessed laundry press   
11                          cloud-based plaid ghost   
12                           twirling rolling stone   
13                          the fragile Troll Twins   
14                                      Red Snapper   
15                                      Red Herring   
16                                  pygmy orderlies   
17                                    pygmy janitor   
18                                 pygmy headhunter   
19                               pygmy witch lawyer   
20                         hopping-mad pygmy shaman   
21                                   A.M.C. gremlin   
22                                     tomb servant   
23                                         tomb asp   
24                                         tomb asp   
25                     War Hippy Elite Fire Spinner   
26              yuletide Knob Goblin Assistant Chef   
27  phase-shifting Sub-Assistant Knob Mad Scientist   
28  phase-shifting Sub-Assistant Knob Mad Scientist   
29               clowning Knob Goblin Barbecue Team   
30               Australian Knob Goblin Harem Guard   
31                                     clingy G imp   
32                      broke, ticking Koopa Troopa   
33                                     annoying Zol   
34                                     annoying Zol   
35                         narcissistic Bullet Bill   
36                            The shy Trouser Snake   

                                               \
                                         Type   
1                               BALEFUL HOWL!   
2                    THROW LATTE ON OPPONENT!   
3                               BALEFUL HOWL!   
4   ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
5   ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
6                               BALEFUL HOWL!   
7                               BALEFUL HOWL!   
8   ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
9   ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
10  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
11                     KGB TRANQUILIZER DART!   
12  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
13                   THROW LATTE ON OPPONENT!   
14  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
15                   THROW LATTE ON OPPONENT!   
16  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
17                              BALEFUL HOWL!   
18                             REFLEX HAMMER!   
19                   THROW LATTE ON OPPONENT!   
20  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
21                             REFLEX HAMMER!   
22  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
23                             REFLEX HAMMER!   
24  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
25  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
26                              BALEFUL HOWL!   
27                              BALEFUL HOWL!   
28                              BALEFUL HOWL!   
29                              BALEFUL HOWL!   
30                     KGB TRANQUILIZER DART!   
31                              BALEFUL HOWL!   
32  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   
33                              BALEFUL HOWL!   
34                              BALEFUL HOWL!   
35                     KGB TRANQUILIZER DART!   
36  ASDON MARTIN: SPRING-LOADED FRONT BUMPER!   

                                          
                           Location Turn  
1    

In [437]:
extractBanishesAndSniffs(dDict,'sniffs')

Day #1                            \
                          Monster                      Type   
1      animated ornate nightstand            PERCEIVE SOUL!   
2      animated ornate nightstand  OFFER LATTE TO OPPONENT!   
3        unlucky baa-relief sheep            PERCEIVE SOUL!   
4                 huge slick lihc            PERCEIVE SOUL!   
5              frozen gaunt ghuol            PERCEIVE SOUL!   
6              drunk Quiet Healer            PERCEIVE SOUL!   
7   the annoying Bubblemint Twins  OFFER LATTE TO OPPONENT!   
8        narcissistic Bob Racecar            PERCEIVE SOUL!   
9        narcissistic Bob Racecar  OFFER LATTE TO OPPONENT!   
10              Green Ops Soldier            PERCEIVE SOUL!   
11                 frozen Blooper            PERCEIVE SOUL!   

                                          
                           Location Turn  
1               The Haunted Bedroom   27  
2               The Haunted Bedroom   27  
3                 The Hidden Temple   75  
4                 The Defiled Niche   83  
5                The Defiled Cranny  156  
6   The Penultimate Fantasy Airship  173  
7                         Twin Peak  250  
8              Inside the Palindome  352  
9              Inside the Palindome  352  
10   The Battlefield (Frat Uniform)  437  
11                      8-Bit Realm  500

In [438]:
extractBanishesAndSniffs(dDict,'macros')

Day #1                   \
                          Monster             Type   
1                 huge slick lihc  MACROMETEORITE!   
2              ticking slick lihc  MACROMETEORITE!   
3         invisible oasis monster  MACROMETEORITE!   
4   the annoying Bubblemint Twins  MACROMETEORITE!   
5                      slime blob  MACROMETEORITE!   
6                      slime blob  MACROMETEORITE!   
7                      slime blob  MACROMETEORITE!   
8            disguised slime blob  MACROMETEORITE!   
9          left-handed slime blob  MACROMETEORITE!   
10              vegetable gremlin  MACROMETEORITE!   

                                                           \
                                            Location Turn   
1                                  The Defiled Niche   83   
2                                  The Defiled Niche   86   
3                                          The Oasis  242   
4                                          Twin Peak  250   
5                                       Sonofa Beach  304   
6                                       Sonofa Beach  315   
7                                       Sonofa Beach  326   
8                                       Sonofa Beach  337   
9                                       Sonofa Beach  348   
10  Next to that Barrel with Something Burning in it  370   

                                             
                                    NextMon  
1                         a dirty old lihc!  
2                         a dirty old lihc!  
3                                   a blur!  
4    an elephant (meatcar?) topiary animal!  
5                         a lobsterfrogman!  
6                         a lobsterfrogman!  
7                         a lobsterfrogman!  
8                         a lobsterfrogman!  
9                         a lobsterfrogman!  
10                     a batwinged gremlin!

In [439]:
def extractWishes(sessionLogs):
    ''' Quick function to ascertain wish usage '''
    
    wishDict = {}
    
    for day in sessionLogs.keys():
        
        wishDict[day] = {}
        
        # Wishes aren't quite as easy as pulls, which are easily snagged
        #   on one line with newline split logic, but they're certainly 
        #   easier than digitizations.
        
        # Combat parsing, utilizing the combat to pull monsters out
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "genie summoned monster" in row:
                turn = row[(row.find('[')+1):row.find(']')]
                wishMon = sessionLogs[day].split('\n')[ct+1].replace('Encounter: ','')
                wishMon = cleanNames(wishMon)
                wishDict[day][ct] = {'Type':'Fight',
                                     'Details':wishMon,
                                     'Turn':int(turn)}
        
        # Extra parsing, using the actual wish URL to fill others
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "&wish=" in row:
                wishString = row[(row.find('&wish=')+6):]
                try: 
                    
                    # Note we have to use ct+2 due to session logs
                    #   taking 2 lines to get from URL to fight. If
                    #   this ever changes, will need to change.
                    
                    wishDict[day][ct+2]['Type']
                    
                except:
                    
                    wishDict[day][ct] = {'Type':'Non-Fight',
                                         'Details':wishString,
                                         'Turn':'?'}

        # Rename the wishes from line # to wish #
        for i, val in enumerate(wishDict[day]):
            wishDict[day][i+1] = wishDict[day].pop(val)
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wishDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wishDict.keys()]).T

extractWishes(dDict)

Day #1                        
        Type            Details Turn
1      Fight       mountain man  137
2      Fight  Green Ops Soldier  410
3  Non-Fight   to+be+Dirty+Pear    ?

In [4]:
def extractPulls(sessionLogs):
    ''' Quick function to extract normal run pulls into a table. '''
    
    pullDict = {}
    
    for day in sessionLogs.keys():
        
        # Find all your pulls; relies on sessions storing them as 'pull: '
        pulls = [i[6:] for i in sessionLogs[day].split('\n') if i.startswith('pull: ')]
        
        # Split out pulls into a 20 item list, for formatting
        numbs = [int(i[0:2].strip()) for i in pulls]
        pullDict[day] = []
        for count, pull in enumerate(pulls):
            pullDict[day] = pullDict[day] + [pull[1:].strip()]*numbs[count]
        
        if len(pullDict[day]) < 20:
            pullDict[day] = pullDict[day] + ['']*(20-len(pullDict[day]))
    
    # Renaming for simplicity/ease of pasting; 
    pullsOut = pd.DataFrame(pullDict).rename(
        index  = lambda x: '#{}'.format(x+1),
        columns= lambda x: 'Day #{}'.format(x))
    
    return pullsOut

extractPulls(dDict)

,Day #1
#1,vampagne
#2,vampagne
#3,vampagne
#4,odor extractor
#5,odor extractor
#6,spice melange
#7,Ultra Mega Sour Ball
#8,deck of lewd playing cards
#9,muculent machete
#10,facsimile dictionary


In [441]:
def goblinParser(sessionLogs):
    ''' Figure out where/when the user encountered sausage goblins.
        This SHOULD NOT be used as a spading tool, as it does not
        properly track important things like how much meat was in
        the grinder and how many turns the grinder was used. It's
        just for me to think about where I dropped goblins in-run '''
    
    goblinDict = {}
    gobNum = 0
    
    for day in sessionLogs.keys():
        
        # I am keeping this a list in case we eventually find a 
        #   sausage goblin boss...
        
        for mon in ['sausage goblin']:
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    gobNum = gobNum + 1
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        # print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        # Gotta show that error!
                        print('  Could not parse {}'.format(data))
                    
                    goblinDict[gobNum] =      {'Turn': turn, 
                                               'Location': loct,
                                               'Day': day}
    
        
    return pd.DataFrame.from_dict(goblinDict).T
                        
goblinParser(dDict)

,Day,Location,Turn
1,1,The Spooky Forest,18
2,1,The Spooky Forest,33
3,1,The Neverending Party,96
4,1,The Penultimate Fantasy Airship,110
5,1,The Copperhead Club,316


## Unfinished functions, to be added over time

In [19]:
def buffSelection(sessionLogs):
    ''' Parse out the IOTM buffs selected in the selected run, by day/turn '''

In [20]:
def itemSelection(sessionLogs):
    ''' Parse out the IOTM items selected in the selected run, by day '''

In [442]:
def zoneInformation(mafEnc,mafCom):
    ''' Parser that turns Mafia's encounter dictionary & monster directory into a
          pythonic dictionary that can be accessed for monsters in each zone. Used
          by the parser to figure out what was a noncombat & what was a fight. '''
    
    zInfo = {}
    
    # Parse through all the monster buckets for each zone
    for zText in mafCom.text.split('\n'):
        if   zText in ['', '1']:     continue # handle weird null rows 
        elif zText[0]=='#':          continue # handle comments
        else:
            # Format ======>  zone \t combat% \t monsters 
            zName = zText.split('\t')[0]
            zCPCT = zText.split('\t')[1]
            zMons = zText.split('\t')[2:]
            
            for n, mon in enumerate(zMons):
                # Remove mafia information about boss status & combat weighting (for now) 
                if ':' in mon: 
                    nMon = mon.split(': ')[0]
                    zMons[n] = nMon
                    
            zInfo[zName] = {'cPercent':zCPCT, 'mons':zMons, 'semirare':[], 'clover':[]}
            
    # Parse through all NC encounters in each zone
    for zText in mafEnc.text.split('\n'):
        if   zText in ['', '1']:     continue # handle weird null rows 
        elif zText[0]=='#':          continue # handle comments
        else: 
            # Format ======>  zone \t encounter type \t title 
            zName = zText.split('\t')[0]
            zType = zText.split('\t')[1]
            zEnct = zText.split('\t')[2]
            
            # Just trying to find clovers & semirares
            if   zType == 'CLOVER':   zInfo[zName]['clover']   = zInfo[zName]['clover'] + [zEnct]
            elif zType == 'SEMIRARE': zInfo[zName]['semirare'] = zInfo[zName]['semirare'] + [zEnct]
    
    return zInfo
            
zoneInfo = zoneInformation(mafEncounters,mafCombats)

In [414]:
class newLogParser():
    ''' Giant class that builds a whole damn new log. '''
    
    def __init__(self, runDates = ['20190505','20190506'],
            kolmafDir = '/Users/amcguire/Library/Application Support/KoLmafia/',
            outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/',
            kolName   = 'Captain Scotch',
            zInfo     = {}):
        
        # Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
        self.runDates   = runDates
        self.kolmafDir  = kolmafDir
        self.outputDir  = outputDir
        self.kolName    = kolName
        self.zInfo      = zInfo
        
        # Set universal wordbuckets for parsing purposes
        self.musWords = ['Beefiness','Fortitude','Muscleboundness','Strengthliness','Strongness']
        self.mysWords = ['Enchantedness','Magicalness','Mysteriousness','Wizardliness']
        self.moxWords = ['Cheek','Chutzpah','Roguishness','Sarcasm','Smarm']
        
        # initialize a bunch of variables we refer back to & modify
        self.test         = False  # If false, ignores linesToParse
        self.linesToParse = 5000   # Lines to parse; testing function
        
        self.turnSpent    = 0      # How many turns have actually been spent
        self.turnSeen     = -1     # How many "turns" mafia has seen; starts at -1 to bypass valhalla 
        self.lineIndex    = 0      # How many lines it has read
        self.lineIndexPre = 0      # So I can do the line range function; initialize
        self.currDay      = 0      # The current day
        self.newEncounter = False  # This lets the script build its buffer
        self.finalLog     = {}     # The actual final log, w/ "turns seen" as the index
        self.advString    = ''     # Build the adventure string for parsing
        self.finalBuffer  = {}     # The final buffer log, for me to do later parsing. Much more painful.
        self.buffer       = ''     # For storing b/w adventure stuff for later parsing
        
        # Checking mafia's free action counter
        self.freeNC       = 0
        self.freeMafC     = 0
        self.freeMafMiss  = 0
        
        # Initial support for the mayfourth saber friend parsing
        self.saberFriend  = ['']     # What the current Saber Friend is
        self.saberCount   = 0      # How many Saber Friends have been encountered; resets @ 2
        self.saberUsage   = 0      # How many actual fires of the Saber have been done
        
    def preProcessing(self, sLogs):
        ''' In case I want to apply preprocessing in the future. This will
              likely remove bits of the buffer I am 100% sure I do not need. 
              But I haven't really touched the buffer yet, so... '''
        
        for key, day in enumerate(sLogs):
        
            # This new encounter string is *really* messing stuff up. 
            sLogs[key+1] = sLogs[key+1].replace('Encounter: Using the Force','USED THE FORCE')
        
        
        return sLogs 
    
    def readSessions(self):
        ''' Subfunction that reads in your sessions into dictionary format '''
        
        dDict = {}
        
        for day, session in enumerate(self.runDates):
            # Python is zero-indexed, so we increment day by 1 here. 
            #   Also replace spaces w/ _ for playername.

            dDict[day+1] = open('{}sessions/{}_{}.txt'.format(self.kolmafDir, self.kolName.replace(" ","_"), session)).read()

            # Remove everything before Valhalla & after freeing Ralph
            if day == 0:
                start = dDict[1].find('Welcome to Valhalla!')
                dDict[1] = '[0] '+dDict[1][start:]

            else:
                end   = dDict[day+1].find('Freeing King Ralph')
                dDict[day+1] = dDict[day+1][:end]
        
        return dDict
    
    # ================================================================================
    
    def resetAdv(self):
        ''' Reset all adventure conditions after parsing a combat '''
        
        self.combround    = -1     # How many rounds the combat went; -1 for NCs
        self.combat       = False  # Is it a combat, or a NC? False for NC, true for C
        self.ncInfo       = {}     # Dictionary storing NC information
        self.mus          = 0      # mus statgain
        self.mys          = 0      # mys statgain
        self.mox          = 0      # mox statgain
        self.hpLoss       = 0      # hp loss
        self.hpGain       = 0      # hp gain
        self.meat         = 0      # meat gain
        self.itemsUsed    = []     # All items *used* in combat
        self.effGained    = []     # All effects gained in combat (tuple; duration after effect name)
        self.items        = []     # All items gained in combat
        self.skills       = []     # All skills cast in combat
        self.currLoct     = "KoL"  # Current location
        self.advTitle     = "???"  # Adventure title; basically the encounter string
        self.freeStatus   = 0      # Did mafia say it cost a turn?
        self.origMonsters = []     # Store the original monster when you macro
    
    def parseAdventure(self, advTxt=''):
        ''' Big ol' function for parsing adventures. '''
        for ct, row in enumerate(advTxt.split('\n')): 
            
            # Start by finding the location
            if ']' in row:
                if '[' == row[0]:
                    data = advTxt.split('\n')[ct].split('] ')
                    self.currLoct = data[1]
                    try: 
                        line2 = advTxt.split('\n')[ct+1]
                    except:
                        line2 = ''
            
            # Figure out what the adventure title is
            elif 'Encounter:' in row:
                self.advTitle = line2.replace('Encounter: ','')
            
            elif row[0:6] == "Round ":
                self.combround = int(row[6:].split(':')[0])
                
                if '{} casts '.format(self.kolName) in row:
                    self.skills = self.skills + [row.split('{} casts '.format(self.kolName))[1]]
                    
                elif 'hit point' in row:
                    if   row.split(' ')[-4] == 'lose': self.hpLoss = self.hpLoss - int(row.split(' ')[-3].replace(',',''))
                    elif row.split(' ')[-4] == 'gain': self.hpGain = self.hpGain + int(row.split(' ')[-3].replace(',',''))
                
                elif '{} uses '.format(self.kolName) in row:
                    self.itemsUsed = self.itemsUsed + [row.split('{} uses '.format(self.kolName))[1]]
                    
                elif 'your opponent becomes a ' in row:
                    self.origMonsters = self.origMonsters + [self.advTitle]
                    self.advTitle = row.split('your opponent becomes a ')[1][:-1]
                    
            
            elif ('You gain ' in row) & (' Meat' in row):
                self.meat = self.meat + int(row.split('You gain ')[1].replace(' Meat','').replace(',','').replace('.',''))
            
            elif 'After Battle: You gain' in row:
                statString = row.split(' ')[-1]
                
                if   statString in self.musWords: self.mus = self.mus + int(row.split(' ')[-2].replace(',',''))
                elif statString in self.mysWords: self.mys = self.mys + int(row.split(' ')[-2].replace(',',''))
                elif statString in self.moxWords: self.mox = self.mox + int(row.split(' ')[-2].replace(',',''))
                
                elif 'point' in statString:
                    if   row.split(' ')[-4] == 'lose': self.hpLoss = self.hpLoss - int(row.split(' ')[-3].replace(',',''))
                    elif row.split(' ')[-4] == 'gain': self.hpGain = self.hpGain + int(row.split(' ')[-3].replace(',',''))
                    
            elif 'You acquire an item: ' in row:
                self.items = self.items + [row.split('You acquire an item: ')[1]]
                
            elif 'You acquire an effect: ' in row:
                effName = row.split('You acquire an effect: ')[1].split(' (')[0]
                effLen  = row.split('You acquire an effect: ')[1].split(' (')[1][:-1]
                self.effGained = self.effGained + [(effName,int(effLen))]
                
            elif row == 'This combat did not cost a turn':
                self.freeStatus = 1
            
            elif row == 'Took choice 1387/2: &quot;You will go find two friends and meet me here.&quot;':
                self.saberMonster = [self.advTitle]
                self.saberCount =  0
                self.saberUsage += 1
        
        # Now that it's been fully parsed, figure out if it's a combat or an NC. Some 
        #   zones either A: aren't zones or B: aren't easily parsed. Here's some exclusion 
        #   logic as a wraparound to make sure the whole thing works.
        
        exclZones = ['genie summoned monster','Dr. Gordon Stuart\'s Science Tent',
                     'rusty hedge trimmers', 'Daily Dungeon', 'Typical Tavern Cellar',
                     'Lower Chamber', 'The Hedge Maze']
        
        try: 
            # If the round is -1, it means it never found the "round" text, which = NC.
            if self.combround == -1: 
            
                self.ncInfo = {'combat':'N/A','ncTitle':self.advTitle}

                # Determine clover/semirare status.
                if   self.advTitle in self.zInfo[self.currLoct]['clover']:   self.ncInfo.update({'clover':True})
                elif self.advTitle in self.zInfo[self.currLoct]['semirare']: self.ncInfo.update({'semirare':True})

                # For the purposes of this parser, I will treat the zeppelin NC as a clover. 
                #   The SR/Clover adventures have exactly the same name! Rude.
                elif self.advTitle == 'Methinks the Protesters Doth Protest Too Little': 
                    self.ncInfo.update({'clover':True})
            else: 
                # Figure out if the monster is native to that zone.
                monBkt = self.zInfo[self.currLoct]['mons']
                
                if self.advTitle in monBkt: 
                    self.ncInfo.update({'nativeMonster':True})
                else:
                    self.ncInfo.update({'nativeMonster':np.NaN})
                
                if (self.advTitle == self.saberFriend[0]) & (self.ncInfo['nativeMonster'] == True):
                    self.saberCount = self.saberCount + 1
                    
                    # Flush out friends if you have reached the end of your sabers. 
                    if self.saberCount == 2:
                        self.saberCount  = 0
                        self.saberFriend = ['']
        
        # Now apply those exclusion zones.
        except:
                if any(word in self.currLoct for word in exclZones) or (self.advTitle == '???'):
                    pass
                else: 
                    print('Error at {} in zone {}, at "turn" {}'.format(self.advTitle,self.currLoct, self.turnSeen))
                    #print(advTxt)
                    
        advDict = {'rounds': self.combround, 'combat':self.advTitle, 'mus':self.mus, 'mys':self.mys, 'mox':self.mox,
                   'items':self.items, 'location':self.currLoct,'meat':self.meat, 'free?':self.freeStatus, 'skills':self.skills, 
                   'hpGain':self.hpGain,'hpLoss':self.hpLoss, 'itemsUsed':self.itemsUsed, 'effectsGained':self.effGained,
                   'macroFrom':self.origMonsters}
        
        advDict.update(self.ncInfo)
        
        # Reset adv conditions
        self.resetAdv()
        
        return advDict
    
    def prepareNewLog(self):
        ''' Alright, let's prepare the damn log then. '''
        
        # Read in your sessions w/ the aforementioned function
        sessionLogs = self.readSessions()
        
        # Use preprocessing to initialize needed variables
        sessionLogs = self.preProcessing(sLogs = sessionLogs)
        self.resetAdv()
        
        # Walk through the files provided for session logs
        for d, sess in enumerate(runDates):
            print('Day #{} -- Parsing {}.'.format(d+1, sess))
            
            # When in testing mode, I limited line #s. This disengages the limit if test flag is off.
            if self.test == False:
                self.linesToParse = len(sessionLogs[d+1].split('\n'))
                
                
            # Enumerate through all lines of your session logs
            for ct, row in enumerate(sessionLogs[d+1].split('\n')[0:self.linesToParse]):
                self.lineIndex += 1
                
                if row == '':
                    self.newEncounter = True
                elif row[0] == '[':
                    self.newEncounter = False

                    # ==========================================================
                    # -- This is where it adds the actual new "turn" to the buffer.
                    # --   Each turn is a dictionary that can be easily converted 
                    # --   into a pandas DF. The buffer is all the stuff that 
                    # --   happens between adventures; I'll use this to figure out
                    # --   resource management stuff. Later. 
                    # ==========================================================

                    self.finalBuffer[self.turnSeen] = {
                        'buffer':    self.buffer,
                        'turnSpent': self.turnSpent,
                        'lineRange': [self.lineIndexPre,self.lineIndex]
                    }

                    aDict = self.parseAdventure(advTxt = self.advString)

                    self.finalLog[self.turnSeen] = {
                        'turnSpent': self.turnSpent,
                        'day':d+1,
                        'lineRange': [self.lineIndexPre,self.lineIndex]
                    }

                    self.finalLog[self.turnSeen].update(aDict)

                    self.lineIndexPre = self.lineIndex
                    self.buffer = ''
                    self.advString = ''
                    self.turnSeen  += 1
                    self.turnSpent = row[(row.find('[')+1):row.find(']')]

                if self.newEncounter == True:
                    # Update the buffer if it's buffer time
                    self.buffer = self.buffer + row

                elif self.newEncounter == False:
                    # Create the adv string for parsing it
                    self.advString = self.advString + '\n' + row

        # Apply all post-facto processing 
        self.postProcessing()
        
        colOrder = ['day','turnSpent','advUsed','location','combat', 'free?', 
            'items','meat', 'rounds', 'mox', 'mus', 'mys','effectsGained','clover', 
            'nativeMonster','hpGain', 'hpLoss', 'ncTitle',  'semirare', 'itemsUsed',
            'skills', 'crafting', 'lineRange','freeNC','macroFrom']
        
        pLog = pd.DataFrame(newLog).T.loc[:,colOrder]
        pLog.to_csv('{} -- {}d{}advs.csv'.format(self.kolName,len(runDates),self.turnSpent))
        
        return pLog
    
    def postProcessing(self):
        
        # Remove turn -1 and turnseen = 0 from the log
        self.finalLog.pop(-1)
        self.finalLog.pop(0)
        
        # Add a new 'free' counter that references whether the task took an adv. 
        #   Uses mafia's as base & prints how many "free" items mafia missed.
        
        for t in self.finalLog:
            try: 
                # For free actions, the next row's turnSpent will be equal.
                if self.finalLog[t]['turnSpent'] == self.finalLog[t+1]['turnSpent']:
                    if self.finalLog[t]['rounds'] == -1:
                        self.freeNC += 1
                        self.finalLog[t]['free?'] = 1
                    elif self.finalLog[t]['free?'] == 1:
                        self.freeMafC += 1
                    else:
                        self.freeMafMiss += 1
                        self.finalLog[t]['free?'] = 1    
            except:
                pass
        
        # Report back on how Mafia did!
        print('\nMafia correctly captured {} free combats. It missed {} free/skipped NCs & {} free combats.'.format(self.freeMafC, self.freeNC, self.freeMafMiss))
        
        # How many times did we use the saber?
        print('\nYou used the saber {} times. Embrace the force!'.format(self.saberUsage))
        
        # Rename/refactor crafting so that it's easier to summarize.
        for t in self.finalLog:
            if self.finalLog[t]['location'][0:4] == 'Cook':
                self.finalLog[t]['crafting'] = self.finalLog[t]['location']
                self.finalLog[t]['location'] = 'Crafting Adventure'
            elif self.finalLog[t]['location'][0:4] == 'Mix ':
                self.finalLog[t]['crafting'] = self.finalLog[t]['location']
                self.finalLog[t]['location'] = 'Crafting Adventure'
            # Eventually put smithing here, I don't know how that generates in mafia logs tho.
            
        # Rename/refactor lair stat & damage tests
        elementalTests = ['Coldest', 'Hottest', 'Stinkiest', 'Sleaziest', 'Spookiest']
        statTests = ['Smoothest', 'Strongest', 'Smartest']
        
        for t in self.finalLog:
            if any(test+' Adventurer Contest'==self.finalLog[t]['location'] for test in elementalTests):
                self.finalLog[t]['location'] = '[Element]est Adventurer Contest'
                
            elif any(test+' Adventurer Contest'==self.finalLog[t]['location'] for test in statTests):
                self.finalLog[t]['location'] = '[Offstat]est Adventurer Contest'
        
        # Rename/refactor the Tavern Cellar, Daily Dungeon, & Hedge Maze
        for t in self.finalLog:
            if 'The Typical Tavern Cellar' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Typical Tavern Cellar'
            elif 'Daily Dungeon' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Daily Dungeon'
            elif 'Hedge Maze' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Hedge Maze'
            elif 'Lower Chamber' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Lower Chamber'
        
        # Add a flag for free/skipped NCs so they're easier to filter out 
        for t in self.finalLog:
            if (self.finalLog[t]['rounds'] == -1) & self.finalLog[t]['free?']:
                self.finalLog[t]['freeNC'] = 1
        
        # Add a variable that counts the # of adventures used by that row
        for t in self.finalLog:
            try:
                self.finalLog[t]['advUsed'] = int(self.finalLog[t+1]['turnSpent']) - int(self.finalLog[t]['turnSpent'])
            except:
                self.finalLog[t]['advUsed'] = 1
        
newLog = newLogParser(runDates=['20190506','20190507'],zInfo=zoneInfo).prepareNewLog()

Day #1 -- Parsing 20190506.
Day #2 -- Parsing 20190507.
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 740

Mafia correctly captured 163 free combats. It missed 84 free/skipped NCs & 10 free combats.

You used the saber 10 times. Embrace the force!


In [416]:
newLog.head()

,day,turnSpent,advUsed,location,combat,free?,items,meat,rounds,mox,...,hpGain,hpLoss,ncTitle,semirare,itemsUsed,skills,crafting,lineRange,freeNC,macroFrom
1,1,1,1,Guano Junction,N/A,0,"[sonar-in-a-biscuit, sonar-in-a-biscuit]",0,-1,0,...,0,0,How I Wonder What You're At,NaN,[],[],NaN,"[871, 914]",NaN,[]
2,1,2,1,The Haunted Kitchen,skullery maid,0,"[blood bag, bottle of popskull, Skullery Maid'...",179,4,12,...,105,-1,NaN,NaN,[],"[SING ALONG!, DARK FEAST!]",NaN,"[914, 943]",NaN,[]
3,1,3,1,The Outskirts of Cobb's Knob,Knob Goblin Assistant Chef,0,"[chef's hat, Gathered Meat-Clip]",27,2,8,...,0,0,NaN,NaN,[],[],NaN,"[943, 963]",NaN,[]
4,1,4,0,The Outskirts of Cobb's Knob,Sub-Assistant Knob Mad Scientist,1,[],0,1,0,...,0,0,NaN,NaN,[],[ENSORCEL!],NaN,"[963, 976]",NaN,[]
5,1,4,0,The Haunted Kitchen,sausage goblin,1,"[magical sausage casing, Cobb's Knob map]",378,3,16,...,12,-1,NaN,NaN,[],"[SING ALONG!, CHILL OF THE TOMB!]",NaN,"[976, 1008]",NaN,[]


In [425]:
def resourcesByZone(log):
    ''' Extremely basic function to extract resource usage by zone. '''
    
    # Remove free NCs first
    locSumm = pLog.loc[pLog['freeNC']!=1,:].groupby('location').agg({'free?':[sum,len],'advUsed':[sum]})

    locSumm.columns = ['freeTurns','allTurns','actualAdvs']
    
    banishSkills = ['ASDON MARTIN: SPRING-LOADED FRONT BUMPER!',
                  'REFLEX HAMMER!', 'KGB TRANQUILIZER DART!',
                  'THROW LATTE ON OPPONENT!','BALEFUL HOWL!']
    
    sniffSkills  = ['OFFER LATTE TO OPPONENT!','PERCEIVE SOUL!']
    
    freeKills    = ['ASDON MARTIN: MISSILE LAUNCHER!','CHEST X-RAY!']
    
    locDict = {}
    
    for location in locSumm.index:
        locFull = log.loc[log['location']==location,:]
        
        # Create a list of all items used in the zone, for smoke bombs, enamorangs, & odor extractors.
        locItems = list(locFull['itemsUsed'].apply(pd.Series).stack().reset_index(drop=True))
        
        # Create a list of all skills used in the zone, for sniffs & banishes.
        locSkills = list(locFull['skills'].apply(pd.Series).stack().reset_index(drop=True))
        
        # Count # of sausage goblins in the zone
        gobCount = sum([x=='sausage goblin' for x in locFull['combat'].apply(pd.Series).stack().reset_index(drop=True)])
        
        # Test against the list for skills
        bCount = sum([x in banishSkills for x in locSkills])
        sCount = sum([x in sniffSkills  for x in locSkills])
        fkCount = sum([x in freeKills   for x in locSkills])
        macrCount = sum([x=='MACROMETEORITE!' for x in locSkills])
        sabrCount = sum(['USE THE FORCE' in x for x in locSkills])
        ensorcelCt = sum(['ENSORCEL!' in x for x in locSkills])
        
        # ... and test against the list for items
        gropCount = sum([x=='the green smoke bomb!' for x in locItems])
        enamCount = sum([x=='the LOV Enamorang!' for x in locItems])
        odorCount = sum([x=='the odor extractor!' for x in locItems])
        
        locDict[location] = {'macros':macrCount,
                             'freeKills':fkCount,
                             'banishes':bCount,
                             'sniffs':sCount,
                             'GSBs':gropCount,
                             'enamorangs':enamCount,
                             'odorExtractors':odorCount,
                             'goblinCount':gobCount,
                             'saberCount':sabrCount,
                             'ensorcels':ensorcelCt}

    finalTable = locTable.loc[:,['freeTurns','actualAdvs']].join(pd.DataFrame(locDict).T).sort_values(by='actualAdvs',ascending=False)
    
    return finalTable.loc[:,['actualAdvs','freeTurns','banishes','GSBs','freeKills','saberCount','goblinCount','ensorcels','macros','odorExtractors','sniffs','enamorangs']]

resourcesByZone(newLog).to_csv('resourceSample.csv')

In [6]:
heh = {'a':1,'c':2}
heh.keys()

dict_keys(['a', 'c'])

In [31]:
import random

instances = 100
results = {}
for i in range(instances):
        a = 1000
        b = 1000
        myCounter = 0
        while (a > 0) & (b > 0):
            a -= 36 + random.randrange(1,11)
            b -= 36 + random.randrange(1,11)
            myCounter += 1
        else: 
            if myCounter in results.keys():
                results[myCounter] += 1
            else:
                
                results[myCounter] = 1

print(results)
for a in results:
    print("In {} instances, it required {} stuffing fluffers to kill all frats OR hippies {}% of the time.".format(instances, results[a], results[a]/instances*100))

{24: 60, 25: 39, 23: 1}
In 100 instances, it required 60 stuffing fluffers to kill all frats OR hippies 60.0% of the time.
In 100 instances, it required 39 stuffing fluffers to kill all frats OR hippies 39.0% of the time.
In 100 instances, it required 1 stuffing fluffers to kill all frats OR hippies 1.0% of the time.


In [25]:
import random

results = {}
nInstances = 100

for i in range(nInstances):
    a = 1000
    b = 1000
    myCounter = 0
    
    while (a > 0) & (b > 0):
        a -= 36 + random.randrange(1,11)
        b -= 36 + random.randrange(1,11)
        myCounter += 1
    else:
        try:    results[myCounter] += 1
        except: results[myCounter]  = 1

for numFluffs in results:
    print("In {} instances, it took {} fluffers to kill all frats OR hippies {}% of the time.".format(nInstances,numFluffs,results[numFluffs]*100/nInstances))

In 100 instances, it took 24 fluffers to kill all frats OR hippies 68.0% of the time.
In 100 instances, it took 25 fluffers to kill all frats OR hippies 32.0% of the time.
